In [ ]:
%load_ext autoreload
%autoreload 2

In [15]:
import numpy as np
import requests
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm
from time import sleep
from hydro_serving_grpc.reqstore import reqstore_client

import hydro_serving_grpc as hs_grpc
from client import HydroServingClient, HSServable, HSModel

ImportError: cannot import name 'HSServable' from 'client' (/Users/ygavrilin/Projects/provectus/hydro-root-cause/client.py)

In [2]:
hs_grpc.DataType.Name(hs_grpc.DT_BOOL)

'DT_BOOL'

In [4]:
hs_grpc.DataType.Value("DT_BOOL")

10

In [ ]:
import hydro_serving_grpc as hs_grpc

In [ ]:
from grpc import ssl_channel_credentials

In [ ]:
hs_client = HydroServingClient("dev.k8s.hydrosphere.io", credentials=ssl_channel_credentials())

In [ ]:
mnist_model = hs_client.get_model("mnist")

In [ ]:
mnist_model

In [16]:
import mnist
images = mnist.train_images()

In [21]:
(images[0]/255).tolist()

[[0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0],
 [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0],
 [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0],
 [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0],
 [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0],
 [0.0,
  0.0,
 

Plot random image

In [ ]:
x = images[np.random.randint(0, images.shape[0])]
plt.imshow(x)

Send data to reqstore

In [ ]:
mnist_servable = hs_client.deploy_servable("mnist", 1)
for _ in tqdm(range(10)):
    sleep(0.2)
    x = images[np.random.randint(0, images.shape[0])].reshape((1, 28, 28))
    x = x / 255.
    mnist_servable(x, _profile=True)
mnist_servable.delete()

Check that data is in the reqstore

In [ ]:
rs_client = reqstore_client.ReqstoreClient("localhost:9090", insecure=True)
subsample = list(rs_client.subsampling("1", 10))
image = list(mnist_model.contract.decode_request(subsample[0].entries[0].request).values())[0]
plt.imshow(image[0])

Fetch reqstore descriptors about this image

In [ ]:
folder = mnist_model.id
ts = subsample[0].ts
uid = subsample[0].entries[0].uid

Check reqstore again

In [ ]:
r = rs_client.get(str(folder), ts, uid)
r

Send model description and exlained instance description to rootcause

In [ ]:
r = requests.post("http://localhost/rootcause/rise", json={"model":{"name":"mnist", "version":1},
                                                      "explained_instance":{"timestamp":ts, "uid": uid}})


In [ ]:
r.text

In [ ]:
redirect_url = r.headers['Location']
print(redirect_url)
requests.get(redirect_url).text

In [ ]:
requests.get(redirect_url).text

In [ ]:
import json
import ast

result_id = requests.get(redirect_url).json()['result']
result = requests.get(f"http://localhost/rootcause/fetch_result/rise/{result_id}")
result_response = result.json()
result_response

In [ ]:
masks = np.array(ast.literal_eval(result_response['result']['masks']))

Masks is an array of shape (number of classes, width, heigh). It can be seen as a collection of masks of shape (width, height), one for each class, which show which parts of images activate that particular class.

In [ ]:
masks.shape 

In [ ]:
request = rs_client.get(str(folder), ts, uid).entries[0].request
x = mnist_model.contract.decode_request(request)['input']
# Original image from reqstore
plt.imshow(x[0])

In [ ]:
fig, axes = plt.subplots(1, 10)
fig.set_size_inches(20, 5)
for i in range(10):
    axes[i].imshow(x[0])
    axes[i].imshow(masks[i]/255., alpha = 0.7, cmap = "coolwarm")
    axes[i].axis("off")